Portfolio Utility

In [3]:
import numpy as np
from datetime import datetime
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = cov_matrix


In [ ]:
import pandas as pd

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(-1, 2)
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -364.4973381012146
Enet: -393.7171546924095
Combine: 9.788422088555881
Cenet: -295.0289635023227
Avg: -108.61334031275396
Pc: -23.706772598126904
Pls: -110.39655131376271
Pm: 0.0


Excluding 15 bad predictable industries

In [5]:
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
excluded_industries = ['Food', 'Chems', 'ElcEq', 'Oil', 'Smoke', 'Hshld', 'FabPr', 'Telcm', 'BusEq', 'Beer', 'Hlth', 'Steel', 'Mines', 'Coal', 'Util']
industries = [industry for industry in industries if industry not in excluded_industries]
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = cov_matrix


In [ ]:
import pandas as pd

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(-1, 2)
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -136.3228616768908
Enet: -186.13154002916448
Combine: 10.337319459118849
Cenet: -68.11577917091294
Avg: -23.763423937316936
Pc: -11.331095328526963
Pls: -54.47797186887035
Pm: 0.0


Covariance Shrinkage

In [8]:
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

import pandas as pd
import os
from sklearn.covariance import ledoit_wolf

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

In [ ]:
import pandas as pd

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(-1, 2)
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -460.6507601691957
Enet: -361.6445733130247
Combine: 15.471267376384235
Cenet: -164.71408966198123
Avg: -6.142762738775717
Pc: -14.484779429779548
Pls: -116.26024738980982
Pm: 0.0


Hard constraints - (-0.25, 0.5)

In [11]:
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = cov_matrix


In [ ]:
import pandas as pd

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(-0.25, 0.5)
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: 3.6615145463165994
Enet: -7.262795420507386
Combine: 8.23462142613062
Cenet: -10.750027307979451
Avg: 6.107625207799668
Pc: 13.185488782026313
Pls: 6.805172725681287
Pm: 0.0


No shorting

In [14]:
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = cov_matrix


In [ ]:
import pandas as pd
import numpy as np

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(0, 1)  # No short sale, allocation between 0 and 1
        portfolio_allocation = portfolio_allocation / portfolio_allocation.sum()  # Ensure the sum of allocations is 1 or less
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: 0.07678982773054177
Enet: -0.3304108178487093
Combine: 0.16038175656175635
Cenet: 0.266153313775646
Avg: 0.26745410657043267
Pc: 0.5742508530222106
Pls: 0.37924831442575946
Pm: 0.0


Retail Investor - small funds

In [17]:
import pandas as pd
import os
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = cov_matrix


In [ ]:
import pandas as pd
import numpy as np

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        # Impose the constraints on the portfolio allocation
        portfolio_allocation = portfolio_allocation.clip(0, 0.5)  # No short sale, allocation between 0 and 0.5
        portfolio_allocation = portfolio_allocation / portfolio_allocation.sum()  # Ensure the sum of allocations is 1 or less
        
        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

In [ ]:
import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        
        # Append the sum product to the list
        sum_products.append(sum_product)
    
    # Convert the list of sum products to a NumPy array
    sum_products = np.array(sum_products)
    
    # Calculate the result for the model
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    # Store the result in the dictionary
    results[model] = result

# Calculate the difference between each model and the 'Pm' model, then multiply the result by 1200
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

# Print the differences
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -0.12948967982958604
Enet: -0.5948770202626179
Combine: -0.009337589405799401
Cenet: -0.028616331305602455
Avg: 0.09745914507338395
Pc: 0.3840240243377202
Pls: 0.016699856659027276
Pm: 0.0


-1<w<2 - Shrinkage cov var

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

gamma = 3

portfolio_allocations = {model: pd.DataFrame() for model in models}

merged_data['date'] = merged_data['date'].astype(str)

for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    cov_matrix = cov_matrices[month]
    
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    for model in models:
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(-1, 2) 
        
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])

save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

# Define the gamma value
gamma = 3

# Define the date ranges for splitting
split_date_before = '200112'
split_date_after = '200112'

# Initialize dictionaries to store results for each period
results_full_sample = {}
results_before_split = {}
results_after_split = {}

# Function to calculate utility gain
def calculate_utility_gain(portfolio_allocations, actual_returns, models, gamma):
    results = {}
    for model in models:
        portfolio_allocations_model = portfolio_allocations[model]
        sum_products = []
        for date in portfolio_allocations_model.index:
            weights = portfolio_allocations_model.loc[date].values.flatten()
            actual_returns_date = actual_returns[actual_returns['date'] == date][industries].values.flatten()
            sum_product = np.dot(weights, actual_returns_date.T)
            sum_products.append(sum_product)
        sum_products = np.array(sum_products)
        result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
        results[model] = result
    return results

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -460.65
Enet: -361.64
Combine: 15.47
Cenet: -164.71
Avg: -6.14
Pc: -14.48
Pls: -116.26
Pm: 0.0

Utility Gain Differences - Before December 2001:
Ols: -538.92
Enet: -347.61
Combine: 13.69
Cenet: -198.79
Avg: -20.81
Pc: -32.44
Pls: -197.67
Pm: 0.0

Utility Gain Differences - After December 2001:
Ols: -381.24
Enet: -378.91
Combine: 17.4
Cenet: -126.47
Avg: 9.02
Pc: 4.45
Pls: -30.83
Pm: 0.0


In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

gamma = 3

industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)


# Define the industries for which we will use Pm predicted returns
pm_only_industries = [
    'Paper', 'Cnstr', 'Whlsl', 'Chems', 'Trans', 'Servs', 'Food',
    'BusEq', 'FabPr', 'ElcEq', 'Smoke', 'Telcm', 'Steel', 'Hshld',
    'Beer', 'Mines', 'Hlth', 'Oil', 'Coal', 'Util'
]

# List of all models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Dictionary to hold the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert date column to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the current month
    cov_matrix = cov_matrices[month]
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    for model in models:
        if model == 'Pm':
            # If the model is 'Pm', use its predicted returns for all industries
            predicted_returns = merged_data.loc[merged_data['date'] == month, [f'Pm_{industry}' for industry in industries]].iloc[0].values.reshape(-1, 1)
        else:
            # If the model is not 'Pm', use its predicted returns for non-specified industries
            model_returns = merged_data.loc[merged_data['date'] == month, [f'{model}_{industry}' for industry in industries if industry not in pm_only_industries]].iloc[0]
            # And use 'Pm' predicted returns for the specified industries
            pm_returns = merged_data.loc[merged_data['date'] == month, [f'Pm_{industry}' for industry in pm_only_industries]].iloc[0]
            # Combine them into a single array
            predicted_returns = pd.concat([model_returns, pm_returns]).values.reshape(-1, 1)
        
        # Ensure the predicted returns and the inverted covariance matrix have the same order
        predicted_returns_df = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate portfolio allocation
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns_df)
        portfolio_allocation = portfolio_allocation.clip(-1, 2)  # Applying constraints
        
        # Transpose and set index to current month
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]
        
        # Append to the portfolio allocations
        portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])


save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

gamma = 3

results = {}

for model in models:
    portfolio_allocations_model = portfolio_allocations[model]
    
    sum_products = []
    
    for date in portfolio_allocations_model.index:
        weights = portfolio_allocations_model.loc[date].values
        
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        sum_product = np.dot(weights, actual_returns.T)
        
        sum_products.append(sum_product)
    
    sum_products = np.array(sum_products)
    
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    
    results[model] = result

differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}

for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -761.7916458634473
Enet: -207.098127048268
Combine: 6.143595480904845
Cenet: -199.89284498810264
Avg: -127.31742748361755
Pc: -106.46155250926948
Pls: -250.9473001422151
Pm: 0.0


-0.5<w<0.5

In [ ]:

import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-0.5, upper=0.5)

# Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

# Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

# Close the Pandas Excel writer and output the Excel file
writer.close()

import numpy as np

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the results for each model
results = {}

# Loop over each model
for model in models:
    # Get the portfolio allocations for the model
    portfolio_allocations_model = portfolio_allocations[model]
    
    # Initialize a list to store the sum product for each date
    sum_products = []
    
    # Loop over each date in the portfolio allocations DataFrame
    for date in portfolio_allocations_model.index:
        # Get the weights for the date
        weights = portfolio_allocations_model.loc[date].values
        
        # Get the actual returns for the date
        actual_returns = dataMarketRx[dataMarketRx['date'] == date][industries].values
        
        # Calculate the sum product for the date
        sum_product = np.dot(weights, actual_returns.T)
        sum_products.append(sum_product)
    sum_products = np.array(sum_products)
    result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
    results[model] = result
differences = {model: (result - results['Pm']) * 1200 for model, result in results.items()}
for model, difference in differences.items():
    print(f'{model}: {difference}')

Ols: -14.173815607078996
Enet: -15.015996353164095
Combine: 5.903815992996983
Cenet: -14.19647595630137
Avg: 2.839065681969856
Pc: -3.0098769041861075
Pls: -12.86815640236607
Pm: 0.0


# Ledoit_Wolf -1<w<2

In [ ]:

import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-0.25, upper=0.25)

# Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

# Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

# Define the gamma value
gamma = 3

# Define the date ranges for splitting
split_date_before = '200112'
split_date_after = '200112'

# Initialize dictionaries to store results for each period
results_full_sample = {}
results_before_split = {}
results_after_split = {}

# Function to calculate utility gain
def calculate_utility_gain(portfolio_allocations, actual_returns, models, gamma):
    results = {}
    for model in models:
        portfolio_allocations_model = portfolio_allocations[model]
        sum_products = []
        for date in portfolio_allocations_model.index:
            weights = portfolio_allocations_model.loc[date].values.flatten()
            actual_returns_date = actual_returns[actual_returns['date'] == date][industries].values.flatten()
            sum_product = np.dot(weights, actual_returns_date.T)
            sum_products.append(sum_product)
        sum_products = np.array(sum_products)
        result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
        results[model] = result
    return results

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -0.53
Enet: -3.73
Combine: 4.26
Cenet: -3.58
Avg: 3.14
Pc: -0.03
Pls: -3.98
Pm: 0.0

Utility Gain Differences - Before December 2001:
Ols: -1.97
Enet: -8.74
Combine: 2.31
Cenet: -12.92
Avg: 3.17
Pc: -2.77
Pls: -8.45
Pm: 0.0

Utility Gain Differences - After December 2001:
Ols: 1.62
Enet: 1.81
Combine: 6.46
Cenet: 7.45
Avg: 3.26
Pc: 3.05
Pls: 1.13
Pm: 0.0


In [6]:
import numpy as np

# Add 'Random' to the list of models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm', 'Random']

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        if model == 'Random':
            # Generate random allocations within the portfolio boundary
            random_allocations = np.random.uniform(-0.25, 0.25, len(industries))
            
            # Normalize the random allocations so they sum up to 1
            random_allocations = random_allocations / np.sum(random_allocations)
            
            # Ensure that the random allocations vector is a column vector
            random_allocations = random_allocations.reshape(-1, 1)
            
            # Ensure that the random allocations vector and the inverted covariance matrix have the same index
            random_allocations = pd.DataFrame(random_allocations, index=inv_cov_matrix.columns, columns=['returns'])
            
            # Calculate the portfolio allocation for the month for the 'Random' model
            portfolio_allocation = 1/gamma * inv_cov_matrix.dot(random_allocations)
        else:
            # Get the vector of predicted returns for the month for the current model
            predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
            
            # Ensure that the predicted returns vector is a column vector
            predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
            
            # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
            predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
            
            # Calculate the portfolio allocation for the month for the current model
            portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-0.25, upper=0.25)

        # Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

        # Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -0.43
Enet: -1.99
Combine: 3.46
Cenet: -5.53
Avg: 2.17
Pc: 3.91
Pls: -3.27
Pm: 0.0
Random: -2.98

Utility Gain Differences - Before December 2001:
Ols: -1.52
Enet: -1.28
Combine: 3.94
Cenet: -15.4
Avg: 2.12
Pc: 4.39
Pls: -3.08
Pm: 0.0
Random: -5.18

Utility Gain Differences - After December 2001:
Ols: 0.78
Enet: -2.52
Combine: 2.87
Cenet: 6.15
Avg: 2.03
Pc: 3.26
Pls: -3.41
Pm: 0.0
Random: -0.61


# Sample Covariance Matrix: -1<w<2

In [ ]:

import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-0.25, upper=0.25)

# Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

# Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

# Define the gamma value
gamma = 3

# Define the date ranges for splitting
split_date_before = '200112'
split_date_after = '200112'

# Initialize dictionaries to store results for each period
results_full_sample = {}
results_before_split = {}
results_after_split = {}

# Function to calculate utility gain
def calculate_utility_gain(portfolio_allocations, actual_returns, models, gamma):
    results = {}
    for model in models:
        portfolio_allocations_model = portfolio_allocations[model]
        sum_products = []
        for date in portfolio_allocations_model.index:
            weights = portfolio_allocations_model.loc[date].values.flatten()
            actual_returns_date = actual_returns[actual_returns['date'] == date][industries].values.flatten()
            sum_product = np.dot(weights, actual_returns_date.T)
            sum_products.append(sum_product)
        sum_products = np.array(sum_products)
        result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
        results[model] = result
    return results

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -0.43
Enet: -1.99
Combine: 3.46
Cenet: -5.53
Avg: 2.17
Pc: 3.91
Pls: -3.27
Pm: 0.0

Utility Gain Differences - Before December 2001:
Ols: -1.52
Enet: -1.28
Combine: 3.94
Cenet: -15.4
Avg: 2.12
Pc: 4.39
Pls: -3.08
Pm: 0.0

Utility Gain Differences - After December 2001:
Ols: 0.78
Enet: -2.52
Combine: 2.87
Cenet: 6.15
Avg: 2.03
Pc: 3.26
Pls: -3.41
Pm: 0.0


# Ledoit Shrinkage - BEST INDUSTRIES

In [6]:

import pandas as pd
import os
import numpy as np
from sklearn.covariance import ledoit_wolf

industries = ['Books', 'Fin', 'Carry', 'Games', 'Clths', 'Txtls', 'Autos', 'Meals', 'Rtail', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix, _ = ledoit_wolf(dataMarketRx[industries].iloc[start_month:end_month])
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-1, upper=2)

# Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

# Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

# Define the gamma value
gamma = 5

# Define the date ranges for splitting
split_date_before = '200112'
split_date_after = '200112'

# Initialize dictionaries to store results for each period
results_full_sample = {}
results_before_split = {}
results_after_split = {}

# Function to calculate utility gain
def calculate_utility_gain(portfolio_allocations, actual_returns, models, gamma):
    results = {}
    for model in models:
        portfolio_allocations_model = portfolio_allocations[model]
        sum_products = []
        for date in portfolio_allocations_model.index:
            weights = portfolio_allocations_model.loc[date].values.flatten()
            actual_returns_date = actual_returns[actual_returns['date'] == date][industries].values.flatten()
            sum_product = np.dot(weights, actual_returns_date.T)
            sum_products.append(sum_product)
        sum_products = np.array(sum_products)
        result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
        results[model] = result
    return results

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -62.33
Enet: -176.04
Combine: 28.77
Cenet: -64.21
Avg: 25.47
Pc: 4.88
Pls: 12.93
Pm: 0.0

Utility Gain Differences - Before December 2001:
Ols: -17.02
Enet: -204.51
Combine: 29.92
Cenet: -51.71
Avg: 39.28
Pc: 4.41
Pls: 15.13
Pm: 0.0

Utility Gain Differences - After December 2001:
Ols: -110.71
Enet: -145.26
Combine: 27.64
Cenet: -75.1
Avg: 12.12
Pc: 5.68
Pls: 10.56
Pm: 0.0


# Sample Covariance - BEST INDUSTRIES

In [ ]:

import pandas as pd
import os
import numpy as np


industries = ['Books', 'Fin', 'Carry', 'Games', 'Clths', 'Txtls', 'Autos', 'Meals', 'Rtail', 'Other']
industries = [industry.strip() for industry in industries]
dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
merged_data = dataMarketRx
for industry in industries:
    forecastLongShort = pd.read_csv(os.path.join(dir_path, f'forecastLongShort_{industry}_HorizonHorizon01.csv'))
    forecastPm = pd.read_csv(os.path.join(dir_path, f'forecastPm_{industry}.csv'))
    forecastLongShort['date_eval'] = pd.to_datetime(forecastLongShort['date_eval'])
    forecastPm['date_eval'] = pd.to_datetime(forecastPm['date_eval'])
    numeric_cols = forecastLongShort.select_dtypes(include=['number']).columns
    forecastLongShort[numeric_cols] = forecastLongShort[numeric_cols].div(100)
    numeric_cols = forecastPm.select_dtypes(include=['number']).columns
    forecastPm[numeric_cols] = forecastPm[numeric_cols].div(100)
    forecastLongShort.columns = [f"{col}_{industry}" if col != 'date_eval' else col for col in forecastLongShort.columns]
    forecastPm.columns = [f"Pm_{industry}" if 'pHold' in col else col for col in forecastPm.columns]
    merged_data = merged_data.merge(forecastLongShort, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.merge(forecastPm, left_on='date', right_on='date_eval', how='outer')
    merged_data = merged_data.drop(columns=['date_eval_x', 'date_eval_y'])
merged_data['date'] = merged_data['date'].dt.strftime('%Y%m')
cols = ['date'] + [col for col in merged_data if col != 'date']
merged_data = merged_data[cols]
merged_data.to_excel(os.path.join(dir_path, 'merged_data_formatted.xlsx'), index=False)

dir_path = '/Users/mag/Desktop/Thesis/Utilities Portfolio'
dataMarketRx = pd.read_excel(os.path.join(dir_path, 'dataMarketRx.xlsx'))
dataMarketRx['date'] = pd.to_datetime(dataMarketRx['date'], format='%Y%m%d')
dataMarketRx.columns = dataMarketRx.columns.str.strip()
dataMarketRx['date'] = dataMarketRx['date'].dt.strftime('%Y%m')
cov_matrices = {}
for end_month in range(60, len(dataMarketRx)):
    start_month = end_month - 60
    cov_matrix = dataMarketRx[industries].iloc[start_month:end_month].cov()
    cov_matrices[dataMarketRx.iloc[end_month]['date']] = pd.DataFrame(cov_matrix, index=industries, columns=industries)

# Define the models
models = ['Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls', 'Pm']

# Define the gamma value
gamma = 3

# Initialize a dictionary to store the portfolio allocations for each model
portfolio_allocations = {model: pd.DataFrame() for model in models}

# Convert 'date' column in merged_data to string for comparison
merged_data['date'] = merged_data['date'].astype(str)

# Loop over each month in the merged_data DataFrame starting from '198501'
for month in [m for m in merged_data['date'].unique() if m >= '198501']:
    # Get the covariance matrix for the month
    cov_matrix = cov_matrices[month]
    
    # Invert the covariance matrix
    inv_cov_matrix = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    
    # Loop over each model
    for model in models:
        # Get the vector of predicted returns for the month for the current model
        predicted_returns = merged_data[merged_data['date'] == month][[f'{model}_{industry}' for industry in industries]]
        
        # Ensure that the predicted returns vector is a column vector
        predicted_returns = predicted_returns.mean().values.reshape(-1, 1)
        
        # Ensure that the predicted returns vector and the inverted covariance matrix have the same index
        predicted_returns = pd.DataFrame(predicted_returns, index=inv_cov_matrix.columns, columns=['returns'])
        
        # Calculate the portfolio allocation for the month for the current model
        portfolio_allocation = 1/gamma * inv_cov_matrix.dot(predicted_returns)

        portfolio_allocation = portfolio_allocation.clip(lower=-1, upper=2)

# Transpose the portfolio allocation DataFrame and set its index to the date
        portfolio_allocation = portfolio_allocation.T
        portfolio_allocation.index = [month]

# Append the portfolio allocation DataFrame to the corresponding DataFrame in the dictionary
        if portfolio_allocations[model].empty:
            portfolio_allocations[model] = portfolio_allocation
        else:
            portfolio_allocations[model] = pd.concat([portfolio_allocations[model], portfolio_allocation])      

# Specify the directory where you want to save the Excel file
save_dir = '/Users/mag/Desktop/Thesis/Utilities Portfolio'

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(os.path.join(save_dir, 'portfolio_allocations.xlsx'), engine='xlsxwriter')

# Write each DataFrame to a different worksheet
for model, df in portfolio_allocations.items():
    df.to_excel(writer, sheet_name=model)

writer.close()

import numpy as np

# Define the gamma value
gamma = 3

# Define the date ranges for splitting
split_date_before = '200112'
split_date_after = '200112'

# Initialize dictionaries to store results for each period
results_full_sample = {}
results_before_split = {}
results_after_split = {}

# Function to calculate utility gain
def calculate_utility_gain(portfolio_allocations, actual_returns, models, gamma):
    results = {}
    for model in models:
        portfolio_allocations_model = portfolio_allocations[model]
        sum_products = []
        for date in portfolio_allocations_model.index:
            weights = portfolio_allocations_model.loc[date].values.flatten()
            actual_returns_date = actual_returns[actual_returns['date'] == date][industries].values.flatten()
            sum_product = np.dot(weights, actual_returns_date.T)
            sum_products.append(sum_product)
        sum_products = np.array(sum_products)
        result = np.mean(sum_products) - 0.5 * gamma * np.var(sum_products)
        results[model] = result
    return results

# Calculate utility gain for full sample
results_full_sample = calculate_utility_gain(portfolio_allocations, dataMarketRx, models, gamma)

# Filter data for before split
data_before_split = dataMarketRx[dataMarketRx['date'] < split_date_before]
portfolio_allocations_before_split = {model: df[df.index < split_date_before] for model, df in portfolio_allocations.items()}
results_before_split = calculate_utility_gain(portfolio_allocations_before_split, data_before_split, models, gamma)

# Filter data for after split
data_after_split = dataMarketRx[dataMarketRx['date'] > split_date_after]
portfolio_allocations_after_split = {model: df[df.index > split_date_after] for model, df in portfolio_allocations.items()}
results_after_split = calculate_utility_gain(portfolio_allocations_after_split, data_after_split, models, gamma)

# Calculate differences in utility gain for full sample
differences_full_sample = {model: (result - results_full_sample['Pm']) * 1200 for model, result in results_full_sample.items()}

# Calculate differences in utility gain for before split
differences_before_split = {model: (result - results_before_split['Pm']) * 1200 for model, result in results_before_split.items()}

# Calculate differences in utility gain for after split
differences_after_split = {model: (result - results_after_split['Pm']) * 1200 for model, result in results_after_split.items()}

print("Utility Gain Differences - Full Sample:")
for model, difference in differences_full_sample.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - Before December 2001:")
for model, difference in differences_before_split.items():
    print(f'{model}: {round(difference, 2)}')  

print("\nUtility Gain Differences - After December 2001:")
for model, difference in differences_after_split.items():
    print(f'{model}: {round(difference, 2)}') 

Utility Gain Differences - Full Sample:
Ols: -70.74
Enet: -94.64
Combine: 10.45
Cenet: -55.14
Avg: -6.19
Pc: -4.8
Pls: -20.61
Pm: 0.0

Utility Gain Differences - Before December 2001:
Ols: -86.23
Enet: -133.91
Combine: 11.24
Cenet: -58.64
Avg: -10.39
Pc: -10.68
Pls: -47.67
Pm: 0.0

Utility Gain Differences - After December 2001:
Ols: -55.21
Enet: -53.98
Combine: 9.66
Cenet: -51.17
Avg: -1.91
Pc: 1.7
Pls: 7.33
Pm: 0.0


In [19]:
import pandas as pd
import os

# Define the list of industries
industries = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']

# Define the directory
directory = '/Users/mag/Desktop/Thesis/Utility industries seperate'

# Define the list of variables
variables = ['Pm', 'Ols', 'Enet', 'Combine', 'Cenet', 'Avg', 'Pc', 'Pls']

# Initialize a dictionary to store the dataframes
dataframes = {}

# Initialize a dictionary to store the 'Freq' values for each variable
values = {var: [] for var in variables}

# Loop over each industry
for industry in industries:
    # Define the file path
    filepath = os.path.join(directory, f'tablePortfolioSharpeLongShort_{industry}.csv')
    
    # Read the csv file and store it in the dictionary
    dataframes[industry] = pd.read_csv(filepath)
    
    # Append the 'Freq' value for each variable to the respective list
    for var in variables:
        values[var].append(dataframes[industry][dataframes[industry]['Var1'] == var]['Freq'].values[0])

# Calculate and print the cross-sectional averages for each variable
for var in variables:
    average = sum(values[var]) / len(values[var])
    print(f"Cross-sectional average for '{var}': {round(average, 2)}")

Cross-sectional average for 'Pm': 0.31
Cross-sectional average for 'Ols': 0.26
Cross-sectional average for 'Enet': 0.52
Cross-sectional average for 'Combine': 0.48
Cross-sectional average for 'Cenet': 0.57
Cross-sectional average for 'Avg': 0.52
Cross-sectional average for 'Pc': 0.44
Cross-sectional average for 'Pls': 0.57
